# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd 
import requests
import xlsxwriter
import math 


## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [24]:
stock_symbols = pd.read_csv("data/sp_500_stocks.csv")
stock_symbols2 = stock_symbols["Ticker"].sample(n=5, replace=False)
symbols = ",".join(stock_symbols2)
symbols

'STX,CAG,DVN,SWKS,SNPS'

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [25]:
from my_secrets import IEX_CLOUD_API_TOKEN, TWELVEDATA_API_KEY

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [ ]:
base_url = "https://api.twelvedata.com"
symbol = "AAPL"
url = f"{base_url}/quote?symbol={symbols}&apikey={TWELVEDATA_API_KEY}"
# data = requests.get(url).json()


{'STX': {'symbol': 'STX',
  'name': 'Seagate Technology Holdings Plc',
  'exchange': 'NASDAQ',
  'mic_code': 'XNGS',
  'currency': 'USD',
  'datetime': '2025-04-14',
  'timestamp': 1744637400,
  'last_quote_at': 1744660740,
  'open': '75',
  'high': '75.23000',
  'low': '71.60000',
  'close': '72.67000',
  'volume': '3082317',
  'previous_close': '70.15000',
  'change': '2.52000',
  'percent_change': '3.59230',
  'average_volume': '6337902',
  'is_market_open': False,
  'fifty_two_week': {'low': '63.19000',
   'high': '115.32000',
   'low_change': '9.48000',
   'high_change': '-42.65000',
   'low_change_percent': '15.0023734',
   'high_change_percent': '-36.98405',
   'range': '63.189999 - 115.320000'}},
 'CAG': {'symbol': 'CAG',
  'name': 'Conagra Brands Inc.',
  'exchange': 'NYSE',
  'mic_code': 'XNYS',
  'currency': 'USD',
  'datetime': '2025-04-14',
  'timestamp': 1744637400,
  'last_quote_at': 1744660740,
  'open': '26',
  'high': '26.335',
  'low': '25.77000',
  'close': '26.1900

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [ ]:
# price = data["close"]

'202.61'

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [52]:
df1 = pd.read_csv("data/constituents-financials.csv", usecols=["Symbol", "Price", "Market Cap"])
df1.dropna(inplace=True)
df1["Price"].astype(np.float32)
df1.head()

,Symbol,Price,Market Cap
0,MMM,152.20,8.329418e+10
1,AOS,67.30,9.758165e+09
2,ABT,127.93,2.218895e+11
3,ABBV,183.90,3.249770e+11
4,ACN,384.95,2.407785e+11


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [53]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

In [55]:
total_makt_cap = df1["Market Cap"].sum()
df1["position_size_pct"] = df1["Market Cap"].apply(lambda x: x / total_makt_cap)
df1["position_size_pct"].astype(np.float32)
df1["num_shares_to_buy"] = df1[["position_size_pct", "Price"]].apply(
    lambda x: math.floor(float(portfolio_size)*x["position_size_pct"] / x["Price"]), axis=1)
df1.head()

,Symbol,Price,Market Cap,position_size_pct,num_shares_to_buy1,num_shares_to_buy
0,MMM,152.20,8.329418e+10,0.001489,9,9
1,AOS,67.30,9.758165e+09,0.000174,2,2
2,ABT,127.93,2.218895e+11,0.003966,31,31
3,ABBV,183.90,3.249770e+11,0.005809,31,31
4,ACN,384.95,2.407785e+11,0.004304,11,11


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: